# AutoGluon: Multimodal Named Entity Recognition (NER)

## Objective
This notebook demonstrates **multimodal NER** using AutoGluon, combining text and visual/contextual features for improved entity recognition.

## Use Case
Multimodal NER is useful for:
- Document understanding (extracting entities from forms, invoices with layout info)
- Social media analysis (entities in posts with images/metadata)
- Medical records (combining clinical notes with structured patient data)
- Resume parsing (text + document structure + formatting)
- News article analysis (text + images + metadata)

## Key Features
- Combines text tokens with additional contextual features
- Can include visual features (fonts, positions, colors)
- Incorporates structured metadata
- Improves entity recognition accuracy
- Handles complex document layouts

In [ ]:
# Install AutoGluon
!pip install -q autogluon

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import os
import shutil

In [ ]:
# Load dataset
# TODO: Upload your multimodal NER dataset or use URL
# Dataset should have:
# - 'token' column (text tokens)
# - 'label' column (entity tags like B-PER, I-ORG, O)
# - Additional feature columns (e.g., position, font_size, is_bold, etc.)
# - Optional: 'image' column for document images

# Example format:
# token | label | x_pos | y_pos | font_size | is_bold | ...
# John  | B-PER | 100   | 50    | 12        | 1       | ...

# Example: train_data = TabularDataset('path/to/multimodal_ner_data.csv')

train_data = None  # Replace with your data
test_data = None   # Replace with your data

print("Dataset loaded successfully!")
if train_data is not None:
    print(f"Training data shape: {train_data.shape}")
    print("\nColumn types:")
    print(train_data.dtypes)
    print("\nSample data:")
    print(train_data.head(10))

In [ ]:
# Set label column
LABEL = 'label'  # TODO: Replace with your label column name (entity tag)

In [ ]:
# Auto-detect problem type based on label
# Multimodal NER is a token classification task
if train_data is not None and LABEL in train_data.columns:
    # Check if the label is numeric (regression) or categorical (classification)
    if pd.api.types.is_numeric_dtype(train_data[LABEL]):
        # Check if it's continuous or discrete
        unique_ratio = train_data[LABEL].nunique() / len(train_data)
        if unique_ratio > 0.05:  # More than 5% unique values suggests regression
            problem_type = 'regression'
            eval_metric = 'rmse'
        else:
            problem_type = 'classification'
            eval_metric = 'roc_auc'
    else:
        problem_type = 'classification'
        eval_metric = 'roc_auc'
else:
    # Default to classification for NER
    problem_type = 'classification'
    eval_metric = 'roc_auc'

print(f"Problem Type: {problem_type}")
print(f"Evaluation Metric: {eval_metric}")
print("\nNote: Multimodal NER uses text + contextual/visual features.")
if train_data is not None:
    print(f"\nUnique entity labels: {train_data[LABEL].nunique()}")
    print(train_data[LABEL].value_counts())

In [ ]:
# Train the model
# AutoGluon will use all available features (text + contextual)
predictor = TabularPredictor(
    label=LABEL,
    problem_type=problem_type,
    eval_metric=eval_metric,
    path='./autogluon-multimodal-ner-model'
).fit(
    train_data=train_data,
    presets='medium_quality',
    time_limit=900
)

print("Model training completed!")
print("The model learned from text and additional contextual features.")

In [ ]:
# Display and save leaderboard
leaderboard = predictor.leaderboard(test_data, silent=True)
print("\nModel Leaderboard:")
print(leaderboard)

# Save leaderboard to CSV
leaderboard.to_csv('leaderboard.csv', index=False)
print("\nLeaderboard saved to leaderboard.csv")

In [ ]:
# Display and save feature importance
try:
    feature_importance = predictor.feature_importance(test_data)
    print("\nFeature Importance:")
    print(feature_importance)
    print("\nNote: See which features (text, position, formatting) help most.")
    
    # Save feature importance to CSV
    feature_importance.to_csv('feature_importance.csv')
    print("\nFeature importance saved to feature_importance.csv")
except Exception as e:
    print(f"Could not compute feature importance: {e}")

In [ ]:
# Make predictions
if test_data is not None:
    predictions = predictor.predict(test_data)
    print("\nPredictions (Entity Tags):")
    print(predictions.head(20))
    
    # For classification, also show prediction probabilities
    if problem_type == 'classification':
        pred_probs = predictor.predict_proba(test_data)
        print("\nPrediction Probabilities:")
        print(pred_probs.head())
        
    # Example: Show recognized entities in context
    print("\nExample entity recognition with multimodal features:")
    print("Token positions, formatting, and context improve accuracy.")
    print("\nExample:")
    print("Token='John' at top of page, bold, large font -> Likely B-PER (person name)")
    print("Token='CEO' near 'John' -> Supports person entity recognition")

In [ ]:
# Save model artifacts as zip file
model_path = './autogluon-multimodal-ner-model'
zip_filename = 'autogluon_multimodal_ner_model'

if os.path.exists(model_path):
    shutil.make_archive(zip_filename, 'zip', model_path)
    print(f"\nModel artifacts saved to {zip_filename}.zip")
else:
    print("Model path not found. Train the model first.")